In [1]:
import pandas as pd

df = pd.read_csv('/content/customer_support_tickets.csv')

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/customer_support_tickets.csv')

In [4]:
df.shape

(8469, 17)

In [5]:
df.columns

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'Product Purchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='object')

In [6]:
df.isnull().sum()

,0
Ticket ID,0
Customer Name,0
Customer Email,0
Customer Age,0
Customer Gender,0
Product Purchased,0
Date of Purchase,0
Ticket Type,0
Ticket Subject,0
Ticket Description,0


In [7]:
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [16]:
df['cleaned_message'] = df['Ticket Description']

In [17]:
df['cleaned_message'] = df['Ticket Description'].str.lower()

In [18]:
df['cleaned_message'] = df['cleaned_message'].str.replace(
    '[^a-zA-Z ]', '', regex=True
)

In [19]:
df[['Ticket Description', 'cleaned_message']].head()

,Ticket Description,cleaned_message
0,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
1,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
2,I'm facing a problem with my {product_purchase...,im facing a problem with my productpurchased t...
3,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
4,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...


In [20]:
issue_keywords = {
    'PAYMENT': ['payment', 'paid', 'card', 'upi', 'transaction'],
    'LOGIN': ['login', 'password', 'signin', 'sign in', 'otp'],
    'DELIVERY': ['delivery', 'delivered', 'shipping', 'order not received'],
    'REFUND': ['refund', 'returned', 'return', 'money back'],
    'BUG': ['bug', 'error', 'issue', 'crash', 'not working']
}

In [21]:
def classify_issue(message):
    for issue, keywords in issue_keywords.items():
        for word in keywords:
            if word in message:
                return issue
    return 'GENERAL'

In [22]:
df['issue_type'] = df['cleaned_message'].apply(classify_issue)

In [23]:
df[['cleaned_message', 'issue_type']].head()

,cleaned_message,issue_type
0,im having an issue with the productpurchased p...,BUG
1,im having an issue with the productpurchased p...,BUG
2,im facing a problem with my productpurchased t...,GENERAL
3,im having an issue with the productpurchased p...,BUG
4,im having an issue with the productpurchased p...,DELIVERY


In [24]:
df['issue_type'].value_counts()

,count
issue_type,
BUG,7066
LOGIN,444
GENERAL,350
PAYMENT,251
REFUND,239
DELIVERY,119


In [25]:
priority_keywords = {
    'P0': ['system down', 'app down', 'not working', 'crash', 'urgent', 'immediately'],
    'P1': ['failed', 'error', 'issue', 'unable', 'problem'],
    'P2': ['delay', 'late', 'slow', 'pending'],
}

In [26]:
def assign_priority(message):
    for priority, keywords in priority_keywords.items():
        for word in keywords:
            if word in message:
                return priority
    return 'P3'

In [27]:
df['priority'] = df['cleaned_message'].apply(assign_priority)

In [28]:
df[['cleaned_message', 'priority']].head()

,cleaned_message,priority
0,im having an issue with the productpurchased p...,P1
1,im having an issue with the productpurchased p...,P1
2,im facing a problem with my productpurchased t...,P1
3,im having an issue with the productpurchased p...,P1
4,im having an issue with the productpurchased p...,P1


In [29]:
df['priority'].value_counts()

,count
priority,
P1,7684
P0,745
P3,32
P2,8


In [30]:
from datetime import datetime, timedelta

In [31]:
sla_hours_map = {
    'P0': 2,
    'P1': 6,
    'P2': 24,
    'P3': 72
}

In [32]:
df['sla_hours'] = df['priority'].map(sla_hours_map)

In [33]:
current_time = datetime.now()

In [34]:
# Due Time = Current Time + SLA Hours

In [35]:
df['due_time'] = df['sla_hours'].apply(
    lambda x: current_time + timedelta(hours=x)
)

In [36]:
df[['priority', 'sla_hours', 'due_time']].head()

,priority,sla_hours,due_time
0,P1,6,2025-12-26 18:22:28.726421
1,P1,6,2025-12-26 18:22:28.726421
2,P1,6,2025-12-26 18:22:28.726421
3,P1,6,2025-12-26 18:22:28.726421
4,P1,6,2025-12-26 18:22:28.726421


In [37]:
issue_report = df['issue_type'].value_counts()
issue_report

,count
issue_type,
BUG,7066
LOGIN,444
GENERAL,350
PAYMENT,251
REFUND,239
DELIVERY,119


In [38]:
priority_report = df['priority'].value_counts()
priority_report

,count
priority,
P1,7684
P0,745
P3,32
P2,8


In [39]:
print("Tickets by Issue Type done by Eben:\n")
print(issue_report)

print("\nTickets by Priority Level done by Eben:\n")
print(priority_report)

Tickets by Issue Type done by Eben:

issue_type
BUG         7066
LOGIN        444
GENERAL      350
PAYMENT      251
REFUND       239
DELIVERY     119
Name: count, dtype: int64

Tickets by Priority Level done by Eben:

priority
P1    7684
P0     745
P3      32
P2       8
Name: count, dtype: int64


In [40]:
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,...,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating,cleaned_message,issue_type,priority,sla_hours,due_time
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,...,Critical,Social media,2023-06-01 12:15:36,NaN,NaN,im having an issue with the productpurchased p...,BUG,P1,6,2025-12-26 18:22:28.726421
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,...,Critical,Chat,2023-06-01 16:45:38,NaN,NaN,im having an issue with the productpurchased p...,BUG,P1,6,2025-12-26 18:22:28.726421
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,...,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0,im facing a problem with my productpurchased t...,GENERAL,P1,6,2025-12-26 18:22:28.726421
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,...,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0,im having an issue with the productpurchased p...,BUG,P1,6,2025-12-26 18:22:28.726421
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,...,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0,im having an issue with the productpurchased p...,DELIVERY,P1,6,2025-12-26 18:22:28.726421


In [41]:
df.to_csv('final_ticket_report.csv', index=False)